In [ ]:
pip install selenium pandas openpyxl
pip install webdriver_manager
pip install openpyxl Pillow

In [ ]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager
import pandas as pd
import time
from bs4 import BeautifulSoup
from openpyxl import load_workbook
from openpyxl.drawing.image import Image as OpenpyxlImage
import os

from PIL import Image as PILImage
import requests
from io import BytesIO
import hashlib
import os

import json

In [ ]:



# 初始化WebDriver
driver = webdriver.Chrome()

# 访问目标网页
driver.get("https://www.fujifilm.com/fbsg/en/products/sg-office-printers")

# JavaScript代码：为整个文档添加点击事件监听器，捕获点击的元素信息
click_listener_script = """
document.addEventListener('click', function(event) {
    var elem = event.target;
    var elemDetails = {
        tagName: elem.tagName,
        id: elem.id,
        className: elem.className,
        textContent: elem.textContent.trim().substring(0, 100) // 获取文本内容的前100个字符，避免内容过长
    };
    window.clickedElementDetails = JSON.stringify(elemDetails); // 将元素信息转换为JSON字符串并存储
}, true);
"""

# 执行JavaScript代码以设置点击事件监听器
driver.execute_script(click_listener_script)

# 在Python中定期检查是否有点击事件发生，并获取被点击元素的信息
try:
    while True:
        # 使用execute_script获取存储的被点击元素信息
        clicked_elem_details = driver.execute_script("return window.clickedElementDetails;")
        if clicked_elem_details:
            print("被点击的元素信息：", clicked_elem_details)
            # 清除存储的信息以避免重复打印
            driver.execute_script("window.clickedElementDetails = '';")
        
        time.sleep(1)  # 每秒检查一次
except KeyboardInterrupt:
    print("停止检查点击事件。")

# 关闭WebDriver
driver.quit()


圖片下載儲存

In [1]:

def download_image(image_url):
    """下载图片，返回本地图片路径。"""
    # 生成图片URL的哈希作为文件名
    hash_object = hashlib.md5(image_url.encode())
    filename = hash_object.hexdigest() + '.png'
    filepath = os.path.join('images', filename)  # 假设所有图片都保存在当前目录的images文件夹下
    
    # 检查图片是否已经下载
    if not os.path.exists(filepath):
        try:
            response = requests.get(image_url)
            response.raise_for_status()  # 确保请求成功
            with open(filepath, 'wb') as f:
                f.write(response.content)
        except requests.exceptions.RequestException as e:
            print(f"Error downloading image {image_url}: {e}")
            filepath = None
    return filepath


給網頁設置點選監聽

In [ ]:

click_listener_script = """
    // 初始化clickedElementDetails为一个空数组，用于存储被点击元素的详细信息
    window.clickedElementDetails = '';

    document.addEventListener('mousedown', function(event) {
        var elem = event.target; // 获取被点击的目标元素
        var elemDetails = {
            tagName: elem.tagName,
            id: elem.id,
            className: elem.className,
            href: elem.href,
            textContent: elem.textContent.trim() .substring(0, 100)
        };
        // 根据元素的类型添加额外的逻辑
        if (elem.tagName.toLowerCase() === 'li' || elem.tagName.toLowerCase() === 'td') {
            console.log('点击的元素内容：', elem);
            // 将元素信息对象添加到数组中
            elemDetails = {
                tagName: '列表'+elem.tagName,
                id: elem.id,
                className: elem.className,
                href: elem.href,
                textContent: elem.textContent.trim() .substring(0, 100)
            };
            
        }
        // 将元素信息对象添加到数组中
        window.clickedElementDetails+=JSON.stringify(elemDetails);
    }, true);
    """



def setup_click_listener_and_capture_details(driver, url):
    """
    访问给定的URL，并在页面上设置点击事件监听器，捕获点击的元素信息。
    """
    # 访问目标网页
    driver.get(url)

    # JavaScript代码：为整个文档添加点击事件监听器，捕获点击的元素信息
    

    # 执行JavaScript代码以设置点击事件监听器
    driver.execute_script(click_listener_script)

    # 在Python中定期检查是否有点击事件发生，并获取被点击元素的信息
    
    # 限定时间内监听点击事件，这里设为10秒作为示例
    # end_time = time.time() + 10
    new_url=''
    try:
        while True:
            try:
                # 使用execute_script获取存储的被点击元素信息
                clicked_elem_details = driver.execute_script("return window.clickedElementDetails;")
                if clicked_elem_details:
                    print("被点击的元素信息：", clicked_elem_details)
                    if 'href' in clicked_elem_details:
                        elem_dic = json.loads(clicked_elem_details)
                        new_url = elem_dic['href']
                        print(elem_dic['href'])
                        break
                    # 清除存储的信息以避免重复打印
                    driver.execute_script("window.clickedElementDetails = '';")
                time.sleep(1)  # 每秒检查一次
            except Exception as e:
                print("监听点击事件时发生错误：", e)
        setup_click_listener_and_capture_details(driver,new_url)
    except Exception as e:
        print("监听点击事件时发生错误：", e)


    

# 初始化webdriver
# 初始化WebDriver
driver = webdriver.Chrome()
url="https://www.fujifilm.com/fbsg/en/products/sg-office-printers"
setup_click_listener_and_capture_details(driver,url)

抓取富士軟片產品資料

In [2]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager
import pandas as pd
import time
from bs4 import BeautifulSoup
from openpyxl import load_workbook
from openpyxl.drawing.image import Image as OpenpyxlImage
import os
# 假设已经定义了download_image函数

def fetch_and_save_product_details(urls):
    # 初始化webdriver
    service = Service(ChromeDriverManager().install())
    driver = webdriver.Chrome(service=service)
    data = []  # 用於儲存所有產品資訊的列表

    # setup_click_listener_and_capture_details(driver, urls)
    # 遍歷所有URL
    for url in urls:
        driver.get(url)  # 開啟網頁
        time.sleep(10)  # 等待網頁加載

        # 找到包含產品的ul元素
        ul_element = driver.find_element(By.CLASS_NAME, "listing-content__results")
        # 獲取所有產品元素
        products_elements = ul_element.find_elements(By.CSS_SELECTOR, "li[class*='col-md-6']")
        
        # 遍歷每個產品元素
        for product in products_elements:
            try:
                # 提取產品名稱、描述、詳情頁面URL和圖片URL
                name = product.find_element(By.CSS_SELECTOR, "h5[itemprop='name'] a").text
                description_html = product.find_element(By.CSS_SELECTOR, "div[itemprop='description']").get_attribute('innerHTML')
                detail_url = product.find_element(By.CSS_SELECTOR, "h5[itemprop='name'] a").get_attribute('href')
                img_url = product.find_element(By.CSS_SELECTOR, "div.thumbnail a img").get_attribute('src')

                # 使用BeautifulSoup解析描述HTML
                soup = BeautifulSoup(description_html, 'html.parser')
                description_text = soup.get_text(separator='\n').strip()
                parts =[i.replace('\n','') for i in  description_text.split('\n\n\n')]
                description = parts[0]  # 第一行是描述
                # 利用列表推導式和next找到符合特定條件的部分
                printing_speed = next((s for s in parts  if 'Printing Speed' in s ),'')
                paper_size = next((s for s in parts  if 'Paper Size' in s ),'')
                paper_weight = next((s for s in parts  if 'Paper Weight' in s ),'')

                # 將產品資訊添加到data列表
                data.append({
                    'Name': name,
                    'Description': description,
                    'Printing Speed': printing_speed,
                    'Paper Size': paper_size,
                    'Paper Weight:': paper_weight,
                    'Detail URL': detail_url,
                    'Image URL': img_url
                })
            except Exception as e:
                # 處理過程中的異常
                print(f"Error extracting product info:{name} {e}")
                # 若出現異常時將當前產品的基本資訊加入data列表
                data.append({
                    'Name': name,
                    'Description': ''.join(parts),
                    'Printing Speed': '',
                    'Paper Size': '',
                    'Paper Weight': '',
                    'Detail URL': detail_url,
                    'Image URL': img_url
                })

    # 關閉webdriver
    driver.quit()
    
    # 將產品資訊儲存到Excel文件
    file_path = 'products_details.xlsx'
    if os.path.exists(file_path):
        existing_df = pd.read_excel(file_path)
    else:
        existing_df = pd.DataFrame()

    new_df = pd.DataFrame(data)
    updated_df = pd.concat([existing_df, new_df], ignore_index=True)
    updated_df.to_excel(file_path, index=False)
    
    wb = load_workbook(file_path)
    ws = wb.active
    
    # 對每個產品的圖片進行處理
    for index, img_data in enumerate(data, start=1):
        image_url = img_data['Image URL']
        local_image_path = download_image(image_url)  # 下載圖片
        if os.path.exists(local_image_path):  # 檢查圖片是否下載成功
            try:
                img = OpenpyxlImage(local_image_path)
                img_height_px = img.height
                img_height_pt = (img_height_px * 3) / 4  # 將像素轉換為點
                ws.row_dimensions[index+1].height = img_height_pt  # 設置行高
                cell_ref = 'K'+str(index+1)  # 定位到正確的單元格
                ws.add_image(img, cell_ref)  # 插入圖片
            except Exception as e:
                print(f"Error adding image to Excel: {e}")

    # 保存帶有圖片的Excel文件
    wb.save('products_details.xlsx')
    
    print('---------------image reptile ok')


In [ ]:
# 使用示例
urls = ["https://www.fujifilm.com/fbsg/en/products/sg-office-printers"
     ,"https://www.fujifilm.com/fbsg/en/products/sg-multifunction-printers"
     ,"https://www.fujifilm.com/fbsg/en/products/sg-production-print"
     ,"https://www.fujifilm.com/fbsg/en/products/sg-wide-format"
     ,"https://www.fujifilm.com/fbsg/en/products/sg-office-supplies"
     ,"https://www.fujifilm.com/fbsg/en/products/sg-interactive-display"
     ,"https://www.fujifilm.com/fbsg/en/products/sg-webcams-and-conference-cameras"]
[fetch_and_save_product_details(url) for url in urls]

fetch_and_save_product_details("https://www.fujifilm.com/fbsg/en/products/sg-office-printers")

---------------image reptile ok


修改產品抓取程式

In [5]:
from selenium import webdriver
from selenium.webdriver.common.by import By
import time
import json

driver = webdriver.Chrome()

def extract_list_or_table(element):
    """
    递归地从给定的元素中提取列表或表格内容。
    """
    if element.tag_name.lower() == "ul":
        items = element.find_elements(By.TAG_NAME, "li")
        return [extract_list_or_table(item) for item in items]
    elif element.tag_name.lower() == "table":
        return extract_table(element)
    else:
        # 处理普通元素
        img = element.find_elements(By.TAG_NAME, "img")
        img_urls = [img_ele.get_attribute('src') for img_ele in img] if img else []
        return {
            "text": element.text,
            "img_urls": img_urls,
            "href": element.get_attribute('href') if element.tag_name.lower() == "a" else None
        }

def extract_table(table_element):
    """
    提取表格数据，返回一个二维列表。
    """
    rows = []
    for row in table_element.find_elements(By.TAG_NAME, "tr"):
        cols = row.find_elements(By.TAG_NAME, "td")
        if not cols:  # 尝试<th>如果<td>为空
            cols = row.find_elements(By.TAG_NAME, "th")
        rows.append([col.text for col in cols])
    return rows

def fetch_data_from_html(url, class_name, class_sub_name):
    driver.get(url)  # 开启网页
    time.sleep(10)  # 等待网页加载

    # 找到包含产品的ul元素
    ul_element = driver.find_element(By.CLASS_NAME, class_name)
    # 获取所有产品元素
    element = ul_element.find_elements(By.CSS_SELECTOR, f"li[class*='{class_sub_name}']")

    data = []
    # 遍历每个产品元素
    for ele in element:
        try:
            item_data = {}
            # 处理所有子元素
            for child in ele.find_elements(By.XPATH, "./*"):
                extracted_data = extract_list_or_table(child)
                if isinstance(extracted_data, list) or extracted_data.get("text"):
                    # 将提取到的数据添加到item_data中，这里可能需要根据实际情况调整键名
                    item_data.update({
                        "content": extracted_data,
                        "detail_url": extracted_data.get("href"),
                        "img_url": extracted_data.get("img_urls")[0] if extracted_data.get("img_urls") else None
                    })
            data.append(item_data)
        except Exception as e:
            print(e)

    return json.dumps(data, ensure_ascii=False)

# 示例用法
url = "https://www.ibon.com.tw/retail_inquiry.aspx#gsc.tab=0"
class_name = "font16"
class_sub_name = ""
print(fetch_data_from_html(url, class_name, class_sub_name))


[]


In [6]:
from selenium import webdriver
from selenium.webdriver.common.by import By
import time
import json

driver = webdriver.Chrome()

def extract_table_data(table_element):
    """
    提取表格(<table>)数据，并转换成列表的列表。
    """
    rows_data = []
    rows = table_element.find_elements(By.TAG_NAME, 'tr')
    for row in rows:
        cells = row.find_elements(By.TAG_NAME, 'td')
        row_data = [cell.text for cell in cells]
        rows_data.append(row_data)
    return rows_data

def fetch_data_from_html(url, class_name):
    driver.get(url)  # 开启网页
    time.sleep(10)  # 等待网页加载

    # 为了简化示例，这里直接寻找表格元素。如果表格被class_name指定的元素包裹，则需要先找到那个元素。
    table_elements = driver.find_elements(By.TAG_NAME, 'table')
    all_tables_data = []

    for table in table_elements:
        table_data = extract_table_data(table)
        all_tables_data.append(table_data)
    
    # 将数据转换成JSON
    json_data = json.dumps(all_tables_data, ensure_ascii=False)
    return json_data

# 示例用法
url = "https://www.ibon.com.tw/retail_inquiry.aspx#gsc.tab=0"
class_name = "font16"
print(fetch_data_from_html(url, class_name))

[[["", "", "", "搜尋", ""], ["", ""]], [["", ""]], [["", ""]], [["請選擇欲查詢門市 :", "                                \n                                    台北市\n                                \n                                    新北市\n                                \n                                    基隆市\n                                \n                                    宜蘭縣\n                                \n                                    桃園市\n                                \n                                    新竹市\n                                \n                                    新竹縣\n                                \n                                    苗栗縣\n                                \n                                    台中市\n                                \n                                    彰化縣\n                                \n                                    南投縣\n                                \n                                    雲林縣\n                      

In [3]:
from selenium import webdriver
from selenium.webdriver.common.by import By
import time
import json
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

# 目標網址
url = "https://www.google.com/url?q=https%3A%2F%2Fap10.ragic.com%2FFOCUSDGPrint%2Fforms81%2F15%3Fwebview%26webaction%3Dform%26ver%3Dnew%26version%3D2%26pfv1010244%3D%25E7%2599%25BD%25E8%2589%25B2%25E8%25A1%25A3%25E6%259C%258D%2520%2528%25E5%2596%25AE%25E9%259D%25A2%2529&sa=D&sntz=1&usg=AOvVaw0aHfi1ePvd8QPZIuYPkWj7"

# 假設 driver 是你的 WebDriver 實例
original_window = driver.current_window_handle

# 等待新視窗或標籤頁出現
driver.wait.until(EC.number_of_windows_to_be(2))

# 切換到新視窗
new_window = [window for window in driver.window_handles if window != original_window][0]
driver.switch_to.window(new_window)

# 打開目標網址
driver.get(url)
# 等待網頁加載完成
time.sleep(10)

# JavaScript代碼，設置點擊事件監聽器
click_listener_script = """
    // 當某元素被點擊時觸發
    document.querySelector('p.freeValue').addEventListener('click', function() {
        // 假設 actionUrl 是從元素屬性中獲取的目標網址
        var actionUrl = this.getAttribute('data-action');
        
        // 如果目標網址是指定的值，則執行以下操作
        if (actionUrl == 'https://www.ibon.com.tw/retail_inquiry.aspx#gsc.tab=0') {
            console.log(`------------------------------------`);
            // 引入 Puppeteer 庫
            const puppeteer = require('puppeteer');

            // 開始異步函數
            (async () => {
                // 啟動瀏覽器
                const browser = await puppeteer.launch();
                const page = await browser.newPage();

                // 導航到目標頁面
                await page.goto(actionUrl);

                // 等待頁面和必要元素加載
                await page.waitForSelector('p.freeValue'); // 確保門店列表已加載

                // 模擬點擊打開 popupTitleBar
                // 注意：這裡假設 '.banner' 是觸發 popupTitleBar 顯示的元素
                await page.click('.banner');

                // 等待 popupTitleBar 加載
                await page.waitForSelector('.popupTitleBar');

                // 抓取資料
                const storeName = await page.$eval('#lblStore_Name', el => el.innerText);
                const storeNumber = await page.$eval('#lblStore_ID', el => el.innerText);

                console.log(`店名: ${storeName}, 店號: ${storeNumber}`);

                // 根據需要執行其他操作，比如填充資料到另一個頁面的表單

                // 關閉瀏覽器
                await browser.close();
            })();
        }
    });
"""

# 執行 JavaScript 代碼
driver.execute_script(click_listener_script)


# 輸入框(Input Box)：
# 一個位於網頁頂部或中間的輸入框，可能帶有提示文字，但無法直接在其中輸入內容。

# 點擊區域(Clickable Area)：
# 這可能是一個按鈕或任何可以被點擊的區域，當用戶點擊此區域時，會觸發 JavaScript 代碼。

# 網頁(Web View)：
# 當用戶點擊點擊區域後，會打開一個新的瀏覽器視窗或標籤頁，並顯示網頁的目標頁面。這個網頁可能包含一些交互元素和資訊，例如表單、按鈕、文字等。


NameError: name 'driver' is not defined

In [ ]:
# app.py
from flask import Flask, render_template, jsonify

app = Flask(__name__)

@app.route('/')
def home():
    return render_template('index.html')

@app.route('/fetch-stores', methods=['GET'])
def fetch_stores():
    # 這裡將呼叫 Node.js 腳本來執行 Puppeteer 任務，並獲取結果
    # 回傳格式：jsonify({'data': '這裡將是7-11分店資訊'})
    return jsonify({'data': '暫時模擬的7-11分店資訊'})

if __name__ == '__main__':
    app.run(debug=True)


In [3]:

from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.common.exceptions import NoSuchElementException
from urllib.parse import urljoin
import csv
import time
# 初始化 WebDriver
 # 替換為您的 ChromeDriver 路徑
driver = webdriver.Chrome()

# 要爬取的網頁
url = 'https://twcoupon.com/brandshop-%E5%85%A8%E5%AE%B6%E4%BE%BF%E5%88%A9%E5%95%86%E5%BA%97-%E9%9B%BB%E8%A9%B1-%E5%9C%B0%E5%9D%80.html'
driver.get(url)
time.sleep(10)

# 找到所有的超連結元素
links = driver.find_elements(By.CSS_SELECTOR, 'a[title^="前往全家便利商店"]')

# 打開一個新的 CSV 檔案進行寫入
try:
    with open('stores.csv', 'w', newline='', encoding='utf-8') as file:
        writer = csv.writer(file)
        writer.writerow(['Title', 'URL', 'Name', 'Address', 'Phone'])
        
        for link in links:
            try:
                print(link.get_attribute('title'))
                title = link.get_attribute('title')
                href = link.get_attribute('href')
                full_url = urljoin(url, href)
                name = link.find_element(By.CSS_SELECTOR, 'li.name').text
                address = link.find_element(By.CSS_SELECTOR, 'li:nth-of-type(2)').text.replace('地址：', '')
                phone = link.find_element(By.CSS_SELECTOR, 'li:nth-of-type(3)').text.replace('電話：', '')
                writer.writerow([title, full_url, name, address, phone])
            except NoSuchElementException as e:
                print(f"Error finding element:{title} {e}")
            except Exception as e:
                print(f"Unexpected error:{title} {e}")
except IOError as e:
    print(f"Error opening or writing to CSV file: {e}")
except Exception as e:
    print(f"Unexpected error when handling CSV file: {e}")
# 清理並關閉瀏覽器
driver.quit()


前往全家便利商店基隆市分店門市
Error finding element:前往全家便利商店基隆市分店門市 Message: no such element: Unable to locate element: {"method":"css selector","selector":"li.name"}
  (Session info: chrome=122.0.6261.112); For documentation on this error, please visit: https://www.selenium.dev/documentation/webdriver/troubleshooting/errors#no-such-element-exception
Stacktrace:
	GetHandleVerifier [0x00007FF68352AD02+56930]
	(No symbol) [0x00007FF68349F602]
	(No symbol) [0x00007FF6833542E5]
	(No symbol) [0x00007FF6833998ED]
	(No symbol) [0x00007FF683399A2C]
	(No symbol) [0x00007FF68338F13C]
	(No symbol) [0x00007FF6833BBCDF]
	(No symbol) [0x00007FF68338F09A]
	(No symbol) [0x00007FF6833BBEB0]
	(No symbol) [0x00007FF6833D81E2]
	(No symbol) [0x00007FF6833BBA43]
	(No symbol) [0x00007FF68338D438]
	(No symbol) [0x00007FF68338E4D1]
	GetHandleVerifier [0x00007FF6838A6F8D+3711213]
	GetHandleVerifier [0x00007FF6839004CD+4077101]
	GetHandleVerifier [0x00007FF6838F865F+4044735]
	GetHandleVerifier [0x00007FF6835C9736+706710]
	(No

In [4]:
import csv
from selenium import webdriver
from selenium.webdriver.common.by import By
import time
# 初始化 WebDriver
# driver_path = 'path/to/your/chromedriver'  # 替換為您的 ChromeDriver 路徑
driver = webdriver.Chrome()

# 讀取原始 CSV 檔案
input_file = 'stores.csv'  # 原始 CSV 檔案名稱
output_file = 'stores_all.csv'  # 輸出的新 CSV 檔案名稱

with open(input_file, mode='r', encoding='utf-8') as infile, open(output_file, mode='w', newline='', encoding='utf-8') as outfile:
    reader = csv.DictReader(infile)
    fieldnames = reader.fieldnames + ['所在區域', '特殊服務']  # 新增兩個欄位
    writer = csv.DictWriter(outfile, fieldnames=fieldnames)
    writer.writeheader()

    for row in reader:
        url = row['URL']
        driver.get(url)
        # time.sleep(1)
        # 嘗試抓取所需信息，如果找不到元素，則保存為空字符串
        ul_element = driver.find_element(By.CLASS_NAME, "branch_detail")

        try:
            area = ul_element.find_element(By.CSS_SELECTOR, 'li.route').text.replace('所在區域：', '')
        except:
            area = ''
        try:
            service = ul_element.find_element(By.CSS_SELECTOR, 'li:nth-of-type(5)').text.replace('特殊服務：', '')
        except:
            service = ''

        # 更新行信息並寫入新 CSV
        row.update({'所在區域': area, '特殊服務': service})
        print(row)
        writer.writerow(row)

# 清理並關閉 WebDriver
driver.quit()

print("處理完成，輸出文件為：" + output_file)


{'Title': '前往全家便利商店 全家基隆仁四店', 'URL': 'https://twcoupon.com/brandshop-11429-%e5%85%a8%e5%ae%b6%e4%be%bf%e5%88%a9%e5%95%86%e5%ba%97-%e5%9f%ba%e9%9a%86%e5%b8%82-%e5%85%a8%e5%ae%b6%e5%9f%ba%e9%9a%86%e4%bb%81%e5%9b%9b%e5%ba%97.html', 'Name': '全家基隆仁四店', 'Address': '200基隆市仁愛區仁四路52號1樓', 'Phone': '02-24265932', '所在區域': '基隆市 〉仁愛區', '特殊服務': '夯番薯,休憩區'}
{'Title': '前往全家便利商店 全家基隆廟口店', 'URL': 'https://twcoupon.com/brandshop-11428-%e5%85%a8%e5%ae%b6%e4%be%bf%e5%88%a9%e5%95%86%e5%ba%97-%e5%9f%ba%e9%9a%86%e5%b8%82-%e5%85%a8%e5%ae%b6%e5%9f%ba%e9%9a%86%e5%bb%9f%e5%8f%a3%e5%ba%97.html', 'Name': '全家基隆廟口店', 'Address': '200基隆市仁愛區仁德里仁三路33號一樓', 'Phone': '02-24289166', '所在區域': '基隆市 〉仁愛區', '特殊服務': ''}
{'Title': '前往全家便利商店 全家基隆新站前店', 'URL': 'https://twcoupon.com/brandshop-11430-%e5%85%a8%e5%ae%b6%e4%be%bf%e5%88%a9%e5%95%86%e5%ba%97-%e5%9f%ba%e9%9a%86%e5%b8%82-%e5%85%a8%e5%ae%b6%e5%9f%ba%e9%9a%86%e6%96%b0%e7%ab%99%e5%89%8d%e5%ba%97.html', 'Name': '全家基隆新站前店', 'Address': '200基隆市仁愛區忠一路19號', 'Phone': '02-24273646', '所在區域

In [6]:
import csv
import requests
from bs4 import BeautifulSoup

# 讀取原始 CSV 檔案
input_file = 'stores.csv'  # 原始 CSV 檔案名稱
output_file = 'store_all_1.csv'  # 輸出的新 CSV 檔案名稱

with open(input_file, mode='r', encoding='utf-8') as infile, open(output_file, mode='w', newline='', encoding='utf-8') as outfile:
    reader = csv.DictReader(infile)
    fieldnames = reader.fieldnames + ['所在區域', '特殊服務', '所在區域網址']  # 新增欄位
    writer = csv.DictWriter(outfile, fieldnames=fieldnames)
    writer.writeheader()

    for row in reader:
        url = row['URL']
        response = requests.get(url)
        soup = BeautifulSoup(response.text, 'html.parser')

        # 嘗試抓取所需信息
        try:
            area_elements = soup.select('li.route a')
            area = ', '.join([element.get_text(strip=True) for element in area_elements])
            area_href = ', '.join([element['href'] for element in area_elements])
        except:
            area = ''
            area_href = ''

        try:
            service_element = soup.select_one('li:contains("特殊服務：")')
            service = service_element.get_text(strip=True).replace('特殊服務：', '')
        except:
            service = ''

        # 更新行信息並寫入新 CSV
        row.update({'所在區域': area, '特殊服務': service, '所在區域網址': area_href})
        print(row)
        writer.writerow(row)

print("處理完成，輸出文件為：" + output_file)


c:\Users\User\Desktop\爬蟲程式\.conda\Lib\site-packages\soupsieve\css_parser.py:856: FutureWarning: The pseudo class ':contains' is deprecated, ':-soup-contains' should be used moving forward.
  warnings.warn(  # noqa: B028


{'Title': '前往全家便利商店 全家基隆仁四店', 'URL': 'https://twcoupon.com/brandshop-11429-%e5%85%a8%e5%ae%b6%e4%be%bf%e5%88%a9%e5%95%86%e5%ba%97-%e5%9f%ba%e9%9a%86%e5%b8%82-%e5%85%a8%e5%ae%b6%e5%9f%ba%e9%9a%86%e4%bb%81%e5%9b%9b%e5%ba%97.html', 'Name': '全家基隆仁四店', 'Address': '200基隆市仁愛區仁四路52號1樓', 'Phone': '02-24265932', '所在區域': '基隆市, 仁愛區', '特殊服務': '夯番薯,休憩區', '所在區域網址': '/brandshopcity-全家便利商店-基隆市-電話-地址.html, /brandshoparea-全家便利商店-基隆市-仁愛區-電話-地址.html'}
{'Title': '前往全家便利商店 全家基隆廟口店', 'URL': 'https://twcoupon.com/brandshop-11428-%e5%85%a8%e5%ae%b6%e4%be%bf%e5%88%a9%e5%95%86%e5%ba%97-%e5%9f%ba%e9%9a%86%e5%b8%82-%e5%85%a8%e5%ae%b6%e5%9f%ba%e9%9a%86%e5%bb%9f%e5%8f%a3%e5%ba%97.html', 'Name': '全家基隆廟口店', 'Address': '200基隆市仁愛區仁德里仁三路33號一樓', 'Phone': '02-24289166', '所在區域': '基隆市, 仁愛區', '特殊服務': '', '所在區域網址': '/brandshopcity-全家便利商店-基隆市-電話-地址.html, /brandshoparea-全家便利商店-基隆市-仁愛區-電話-地址.html'}
{'Title': '前往全家便利商店 全家基隆新站前店', 'URL': 'https://twcoupon.com/brandshop-11430-%e5%85%a8%e5%ae%b6%e4%be%bf%e5%88%a9%e5%95%86%e5%ba%97-%e5

KeyboardInterrupt: 

In [7]:
pip install lxml

   ---------------------------------------- 0.0/3.9 MB ? eta -:--:--
   ---------------------------------------- 0.0/3.9 MB ? eta -:--:--
   ---------------------------------------- 0.0/3.9 MB ? eta -:--:--
   ---------------------------------------- 0.0/3.9 MB 330.3 kB/s eta 0:00:12
   ---------------------------------------- 0.0/3.9 MB 281.8 kB/s eta 0:00:14
    --------------------------------------- 0.1/3.9 MB 525.1 kB/s eta 0:00:08
   - -------------------------------------- 0.2/3.9 MB 908.0 kB/s eta 0:00:05
   -- ------------------------------------- 0.2/3.9 MB 901.1 kB/s eta 0:00:05
   --- ------------------------------------ 0.3/3.9 MB 1.1 MB/s eta 0:00:04
   ------ --------------------------------- 0.6/3.9 MB 1.6 MB/s eta 0:00:03
   ------ --------------------------------- 0.6/3.9 MB 1.6 MB/s eta 0:00:03
   ------ --------------------------------- 0.7/3.9 MB 1.5 MB/s eta 0:00:03
   --------- ------------------------------ 1.0/3.9 MB 2.0 MB/s eta 0:00:02
   ----------- --------

In [8]:
pip install aiohttp aiofiles



   ---------------------------------------- 0.0/365.3 kB ? eta -:--:--
   ---------- ----------------------------- 92.2/365.3 kB 2.6 MB/s eta 0:00:01
   ---------------------- ----------------- 204.8/365.3 kB 2.5 MB/s eta 0:00:01
   ---------------------------- ----------- 256.0/365.3 kB 2.0 MB/s eta 0:00:01
   ---------------------------------------- 365.3/365.3 kB 2.3 MB/s eta 0:00:00
   ---------------------------------------- 0.0/50.5 kB ? eta -:--:--
   ---------------------------------------- 50.5/50.5 kB 2.7 MB/s eta 0:00:00
   ---------------------------------------- 0.0/76.7 kB ? eta -:--:--
   ---------------------------------------- 76.7/76.7 kB ? eta 0:00:00


In [2]:
import csv
import requests
from bs4 import BeautifulSoup
from concurrent.futures import ThreadPoolExecutor, as_completed
import time

# 使用 Session 提高請求效率
session = requests.Session()

def fetch_and_process(url):
    try:
        response = session.get(url)
        soup = BeautifulSoup(response.text, 'lxml')

        area_elements = soup.select('li.route a')
        area = '\', \''.join([element.get_text(strip=True) for element in area_elements])
        area_href = '\', \''.join([element['href'] for element in area_elements])

        id_element = soup.select_one('li:contains("店號：")')
        id = id_element.get_text(strip=True).replace('店號：', '') if id_element else ''

        service_element = soup.select_one('li:contains("特殊服務：")')
        service = service_element.get_text(strip=True).replace('特殊服務：', '') if service_element else ''
        print({'area': area,'id':id, 'service': service, 'area_href': area_href})
        return {'area': area,'id':id,  'service': service, 'area_href': area_href}
    except Exception as e:
        print(f"Error fetching {url}: {e}")
        return {'area': '','id':'' ,'service': '', 'area_href': ''}

def main(input_file, output_file, max_workers=15):
    with open(input_file, mode='r', encoding='utf-8') as infile, open(output_file, mode='w', newline='', encoding='utf-8') as outfile:
        reader = csv.DictReader(infile)
        fieldnames = reader.fieldnames + ['area', 'id', 'service', 'area_href']
        writer = csv.DictWriter(outfile, fieldnames=fieldnames)
        writer.writeheader()

        with ThreadPoolExecutor(max_workers=max_workers) as executor:
            future_to_url = {executor.submit(fetch_and_process, row['URL']): row for row in reader}
            for future in as_completed(future_to_url):
                row = future_to_url[future]
                result = future.result()
                row.update({'area': result['area'] ,'id': result['id'] , 'service': result['service'], 'area_href': result['area_href']})
                writer.writerow(row)
                print(row)
                time.sleep(1)  # 限制請求頻率

if __name__ == "__main__":
    input_csv = 'stores.csv'
    output_csv = 'stores_all_3.csv'
    main(input_csv, output_csv)


{'area': "基隆市', '安樂區", 'id': '017009', 'service': '夯番薯,休憩區', 'area_href': "/brandshopcity-全家便利商店-基隆市-電話-地址.html', '/brandshoparea-全家便利商店-基隆市-安樂區-電話-地址.html"}
{'area': "基隆市', '安樂區", 'id': '022103', 'service': 'Fami 霜淇淋,夯番薯', 'area_href': "/brandshopcity-全家便利商店-基隆市-電話-地址.html', '/brandshoparea-全家便利商店-基隆市-安樂區-電話-地址.html"}
{'area': "基隆市', '中山區", 'id': '017863', 'service': '夯番薯,休憩區', 'area_href': "/brandshopcity-全家便利商店-基隆市-電話-地址.html', '/brandshoparea-全家便利商店-基隆市-中山區-電話-地址.html"}
{'area': "基隆市', '仁愛區", 'id': '019437', 'service': '夯番薯', 'area_href': "/brandshopcity-全家便利商店-基隆市-電話-地址.html', '/brandshoparea-全家便利商店-基隆市-仁愛區-電話-地址.html"}
{'area': "基隆市', '信義區", 'id': '022465', 'service': 'Fami 霜淇淋,夯番薯,休憩區', 'area_href': "/brandshopcity-全家便利商店-基隆市-電話-地址.html', '/brandshoparea-全家便利商店-基隆市-信義區-電話-地址.html"}
{'area': "基隆市', '中正區", 'id': '014328', 'service': 'Fami 霜淇淋,夯番薯,休憩區,相片立可得', 'area_href': "/brandshopcity-全家便利商店-基隆市-電話-地址.html', '/brandshoparea-全家便利商店-基隆市-中正區-電話-地址.html"}
{'area': "基隆市', '安樂區", 'id'

In [12]:
pip install lxml


In [3]:
import csv
import asyncio
import aiohttp
from bs4 import BeautifulSoup
import aiofiles

# 異步 Session
async def fetch(session, url):
    async with session.get(url) as response:
        return await response.text()

# 異步處理每個 URL
async def fetch_and_process(session, url):
    try:
        html = await fetch(session, url)
        soup = BeautifulSoup(html, 'lxml')

        area_elements = soup.select('li.route a')
        area = ', '.join([element.get_text(strip=True) for element in area_elements])
        area_href = ', '.join([element['href'] for element in area_elements])

        id_elements = soup.select_one('li:-soup-contains("店號：")')
        id = id_elements.get_text(strip=True).replace('店號：', '') if service_element else ''

        service_element = soup.select_one('li:-soup-contains("特殊服務：")')
        service = service_element.get_text(strip=True).replace('特殊服務：', '') if service_element else ''

        return {'area': area ,'id': id , 'service': service, 'area_href': area_href}
    except Exception as e:
        print(f"Error fetching {url}: {e}")
        return {'area': '','id': id , 'service': '', 'area_href': ''}

# 主函式
async def main(input_file, output_file):
    async with aiohttp.ClientSession() as session:
        # 讀取 CSV
        with open(input_file, mode='r', encoding='utf-8') as infile:
            reader = csv.DictReader(infile)
            rows = [row for row in reader]

        # 異步抓取和處理
        tasks = [fetch_and_process(session, row['URL']) for row in rows]
        results = await asyncio.gather(*tasks)

        # 異步寫入 CSV
        async with aiofiles.open(output_file, mode='w', encoding='utf-8', newline='') as outfile:
            writer = csv.DictWriter(outfile, fieldnames=reader.fieldnames + ['area', 'service', 'area_href'])
            await outfile.write(','.join(writer.fieldnames) + '\n')
            for row, result in zip(rows, results):
                row.update(result)
                await outfile.write(','.join([str(row[field]) for field in writer.fieldnames]) + '\n')

# 執行

# 如果是在 Jupyter Notebook 或其他已存在事件循環的環境下執行，使用這個方式來運行主函式
if __name__ == "__main__":
    input_csv = 'stores.csv'
    output_csv = 'stores_all_2.csv'
    loop = asyncio.get_event_loop()
    if loop.is_running():
        # 如果事件循環已經在運行，使用 create_task 或 ensure_future 添加任務
        task = loop.create_task(main(input_csv, output_csv))
    else:
        # 如果事件循環未運行，則像通常一樣運行主函式
        loop.run_until_complete(main(input_csv, output_csv))